In [2]:
import os
import glob
import pickle
from datetime import datetime
import time
import dotenv
import re

import pandas as pd
import pandas_dedupe

import requests
import requests.auth

import praw

import openai

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# load secrets from .env into environment variables
dotenv.load_dotenv()

print(f"{'PRAW:':<20} {praw.__version__ :>10}")
print(f"{'OpenAI:':<20} {openai.version.VERSION :>10}")


PRAW:                     7.7.0
OpenAI:                  0.27.4


See README.md
 - objective is to use OpenAI for named entity extraction to extract all the songs form [this reddit thread](https://www.reddit.com/r/AskReddit/comments/12viv4v/what_is_the_prettiest_song_you_ever_heard_in_your/) and make Spotify playlist
 - use Reddit PRAW API to download all the comments (get [Reddit API key](https://www.reddit.com/prefs/apps))
 - use OpenAI API with a prompt like, extract all the songs from this text to CSV get ([OpenAI API key](https://platform.openai.com/account/api-keys))
 - use Spotify API to make a playlist (get [Spotify API key](https://developer.spotify.com/documentation/web-api/tutorials/getting-started))
 - works, needed a lot of scrubbing, but about 1 day of work, wouldn't have been possible to do a 700-song playlist manually without a team of Mechanical Turks or something
 - If I wanted to go nuts, would process comments individually, save a file for each comment's extracted songs, would make it easier to track down what OpenAI gets wrong, have a resumable, retryable, repeatable process and 
 - Spotify playist is [here](https://open.spotify.com/playlist/08YFkbtTV6GBfNtjJ4PHDu?si=f4761d983ac84091) 
 - needs .env file per dot-env-template
 

In [3]:
# a thread 
submission = "12viv4v"

# minimum karma to process a reply 
minkarma = 5

prompt_prefix="""You will act as a research assistant finding all the artists and track titles from consecutive posts in a reddit thread, and returning them in a CSV format.
Define a CSV format as follows: 
"artist","track"
"The Beatles","Yesterday"
"Eagles","Hotel California" 

You will extract each artist and track referred to in each post delimited by ~~~.
You will generate a list of records containing each artist and track in CSV format. 
The header row should contain `"artist","track"`. 
The fields in each record should be enclosed in double-quotes.
If no artist or track is extracted you will return `No artist and track mentioned in this post.`

"""


# an output file to accumulate all the responses
savefile = 'bronze.txt'

# to speed things we'll cumulate posts til we get to nposts posts or maxchars total chars, whichever comes first
max_post_size=300  # redditor needs to put any songs in 1st couple hundred chars
maxchars = 6000  # max tokens (words/fragments) is 4096 but we'll limit each prompt
nposts = 200 # max posts to combine into a chunk



In [4]:
#  a regular expression to extract values from a csv record, via stackoverflow
csv_validate_re = re.compile(r'''
    \s*                # Any whitespace.
    (                  # Start capturing here.
      [^,"']+?         # Either a series of non-comma non-quote characters.
      |                # OR
      "(?:             # A double-quote followed by a string of characters...
          [^"\\]|\\.   # That are either non-quotes or escaped...
       )*              # ...repeated any number of times.
      "                # Followed by a closing double-quote.
      |                # OR
      '(?:[^'\\]|\\.)*'# Same as above, for single quotes.
    )                  # Done capturing.
    \s*                # Allow arbitrary space before the comma.
    (?:,|$)            # Followed by a comma or the end of a string.
    ''', re.VERBOSE)


## Get all comments from a reddit posting

In [ ]:
def getPraw():
    return praw.Reddit(user_agent="prettiest_song/0.001", 
                       client_id=os.getenv('CLIENT_ID'), 
                       client_secret=os.getenv('CLIENT_SECRET'))


def getAll(r, submissionId, verbose=True):
    submission = r.submission(submissionId)
    submission.comments.replace_more(limit=None)
    commentsList=submission.comments.list()
    return commentsList


In [ ]:
# print(datetime.now())
# r = getPraw()
# res = getAll(r, submission)
# print(datetime.now())

# print("retrieved ", len(res), 'comments')

In [ ]:
# # we have a list of comment objects
# # filter comments with at least some karma
# res3 = [r for r in res if r.score >= minkarma]
# print('filtered to ', len(res3), 'comments')
# res3[0].body, res3[0].score

In [5]:
# save so we can reload it later without downloading

# with open('reddit.pkl', 'wb') as f:
#     pickle.dump(res3, f)
    
with open('reddit.pkl', 'rb') as f:
    res3 = pickle.load(f)
    

## Extract artists and song titles using OpenAI

In [6]:
# check lengths of posts
shorties = []
big_ones = []
for i in range(len(res3)):
    if len(res3[i].body) <3:
        print (i, res3[i].body)
        shorties.append(i)
    if len(res3[i].body) > 4096:
        print(i, len(res3[i].body))
        big_ones.append(i)
        

474 4162
1539 W


In [7]:
# avg length
sum([len(r.body) for r in res3]) / len(res3)

105.34075546719681

In [8]:
print (res3[big_ones[0]].body[:500])

Saturn by Sleeping at Last:
https://www.youtube.com/watch?v=dzNvk80XY9s

The version they did with Tim Fain is even more beautiful: 
https://www.youtube.com/watch?v=0nRpeAiur9Q

I'm not good at choosing one thing from a list of favorites as the best, so I've got about 30+ answers that are really a 30+ -way tie, and the one that I would consider as "prettiest" at any given moment is heavily influenced by my current mood. So, it could be any one of these from my "Heart Wrenchingly Beautiful" playl


In [ ]:
# this doesn't work, gets confused trying to extract post_id and post_score
# prompt_prefix="""You will act as a research assistant finding all the artists and track titles from consecutive posts in a Reddit thread.

# Define a post, delimited by === :
# ===
# post_id: jhbt66b
# post_score: 1549

# I like Yesterday by the Beatles. Also the Eagles - Hotel California got me through some tough times
# ===

# Define a CSV format, delimited by --- :
# ---
# "post_id","artist","track","post_score"
# "jhbt66b","The Beatles","Yesterday",1549
# "jhbt66b","Eagles","Hotel California",1549
# ---

# You will extract all artists and tracks from the following input, and return a list of records containing the post_id, post_score, artist and track extracted from the input in a comma-separated values format. 
# Posts are delimited by ~~~ . 
# The header row should contain `"post_id","post_score","artist","track"`. 
# The input is:"""


In [13]:
openai.api_key = os.getenv('OPENAI_API_KEY')


def get_response(messages, verbose=False):

    prompt = prompt_prefix
    
    for msg in messages:
        prompt += f"""
~~~
{msg}
~~~

"""
    if verbose:
        print(prompt)
        
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo-0301',
        messages=[{"role":"user", 
                  "content": prompt}],
        temperature=0,
    )
    response_msg = response['choices'][0]['message']
    if verbose:
        print(response)
        
    return response_msg['content']



In [14]:
# for each comment object we will extract the body 
# then submit as part of a prompt to chatgpt
print(datetime.now())

slist = res3.copy()
total_posts = len(slist)
print("processing %d posts" % total_posts)

outdir = 'out'
logdir = 'logs'
# make sure out and logs are empty
for f in glob.glob('%s/*' % outdir):
    os.remove(f)
for f in glob.glob('%s/*' % logdir):
    os.remove(f)
count = 0
c = 0

while(slist):  # still comments to process
    chars_to_date = 0
    reply_ids = []
    messages = []

    for _ in range(nposts):  # add up to 100 posts to the prompt
        if slist:
            # make sure no single post > max_post_size, truncate in place as nec 
            slist[0].body = slist[0].body[:max_post_size]
            # total post content < maxchars
            if chars_to_date + len(slist[0].body) < maxchars:
                reply = slist.pop(0)
                reply_ids.append(reply.id)
                body = reply.body
                messages.append(body)
                chars_to_date += len(reply.body)
                c += 1            
            
    # retry loop, have received untrapped 502 error
    RETRIES = 3
    success = False    
    for i in range(RETRIES):
        try:
            response = get_response(messages, verbose=False)
            # no exception thrown
            success=True
            break   
        except Exception as error:
            print("An exception occurred:", error)
            print("Retrying chunk...")
            time.sleep(5)
            continue  # try again
            
    if not success:   # FAIL - retries exhausted
        print('Bailing to next chunk')
        continue

    # do basic validation and cleanup
    # should check first line is valid header and doesn't reverse columns
    csv_valid, csv_err = [], []
    for line in response.split("\n"):
        try:
            csv_values = csv_validate_re.findall(line)
            if len(csv_values) == 2:
                csv_valid.append(line)
            else:
                csv_err.append(line)
        except:
            csv_err.append(line)
    csv_output = "\n".join(csv_valid)
        
    with open("%s/%04d.csv" % (outdir, count), 'w') as outfile:
        outfile.write(csv_output)
    
    if csv_err:
        with open("%s/%04d.err" % (outdir, count), 'w') as outfile:
            outfile.write("\n".join(csv_err))
        
    with open("%s/%04d.log" % (logdir, count), 'w') as logfile:
        logfile.write(str(reply_ids))
        logfile.write('\n\n===== raw prompt =====\n\n')        
        logfile.write("\n=====\n".join(messages))
        logfile.write('\n\n===== raw response =====\n\n')
        logfile.write(response)
        logfile.write('\n\n===== failed validation =====\n\n')
        logfile.write("\n".join(csv_err))
 
    count += 1
#     print(c)
    print(total_posts-len(slist), end=' ')
    
    
print()
print(datetime.now())



2023-05-16 22:47:09.197658
processing 2515 posts
101 201 308 390 466 542 603 659 715 769 823 884 939 1007 1077 1150 1219 1282 1357 1432 1492 1570 1636 1692 1746 1800 1858 1918 1983 2037 2092 An exception occurred: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 239e5d35d6c77ad07b1ee8d085589b44 in your message.)
Retrying chunk...
2159 2210 2259 2304 2355 2403 2459 2505 2515 
2023-05-16 23:32:24.071285


In [20]:
# may still have to tweak the files to get them to load
# should inspect .err files manually and clean up if possible, add additional CSVs

filelist = glob.glob('%s/*.csv' % outdir)

output_df = None

for f in sorted(filelist):
    print(f)
    try:
        tempdf = pd.read_csv("%s" % (f), header=None)
    except Exception as exc:
        print(str(exc))
        continue
    colcount = len(tempdf.columns)
    if len(tempdf.columns) != 2:
        print('%s has %d columns, skipped' % (f, colcount))
        continue
    tempdf.columns=['artist', 'track']
        
    # ok
    # truncate header row
    if tempdf.iloc[0][0]=='artist' and tempdf.iloc[0][1]=='track':
        tempdf = tempdf[1:]
        
    if output_df is not None:        
        output_df = pd.concat([output_df, tempdf], axis=0)
    else:
        output_df = tempdf
        
        
        

out/0000.csv
out/0001.csv
out/0002.csv
out/0003.csv
out/0004.csv
out/0005.csv
out/0007.csv
out/0008.csv
out/0009.csv
out/0010.csv
out/0011.csv
out/0012.csv
out/0013.csv
out/0014.csv
out/0015.csv
out/0016.csv
out/0017.csv
out/0018.csv
out/0019.csv
out/0020.csv
out/0021.csv
out/0023.csv
out/0024.csv
out/0025.csv
out/0026.csv
out/0027.csv
out/0028.csv
out/0029.csv
out/0030.csv
out/0031.csv
out/0032.csv
out/0033.csv
out/0034.csv
out/0035.csv
out/0036.csv
out/0037.csv
out/0038.csv
out/0039.csv


In [21]:
output_df


,artist,track
1,Erik Satie,Gymnopédies
2,Don McLean,"Vincent (Starry, Starry Night)"
3,Lord Huron,The night we met
4,Neil Young,Harvest Moon
5,Simon & Garfunkel,Scarborough Fair
...,...,...
13,Sigur Rós,Valtari
14,Johann Sebastian Bach,"Remember, always finish on de Bach. Never on d..."
15,Apocalyptica,Very cool blend of classical and heavy metal.
16,Simon & Garfunkel,The Boxer


In [22]:
# save bronze
output_df.to_csv(savefile, index=False)
len(output_df)

945

In [23]:
df = pd.read_csv(savefile) \
    .drop_duplicates() \
    .dropna() \
    .sort_values(["artist", "track"]) \
    .reset_index(drop=True)

df.to_csv('silver.csv', index=False)

print(len(df))
# tweak further to get to gold.csv

df


773


,artist,track
0,.Hack//SIGN,Key of the Twilight
1,A Perfect Circle,Gravity
2,A Perfect Circle,Judith
3,A Perfect Circle,The Noose
4,A Perfect Circle,The Nurse who Loved me (cover)
...,...,...
768,Yiruma,River Flows in You
769,Yiruma,Sunbeams... They Scatter
770,Yo La Tengo,My Little Corner of the World
771,beabadoobee,glue song


In [24]:
def fix_leading_trailing(s):
    """First and last should be alphanumeric"""
    # regex prob better if re.match('^\W+(.*)\W+$',playerName): 

    while len(s) and not s[0].isalnum():
        s = s[1:]
        
    while len(s) and not s[-1].isalnum():
        s = s[:-1]
    
    return s.lower().strip()
        

In [25]:
df['artist'] = df['artist'].apply(fix_leading_trailing)
df = df.drop(df.loc[df['artist']=='unknown'].index)
df = df.drop(df.loc[df['artist']=='none'].index)
df = df.drop(df.loc[df['artist']==''].index)
df

,artist,track
0,hack//sign,Key of the Twilight
1,a perfect circle,Gravity
2,a perfect circle,Judith
3,a perfect circle,The Noose
4,a perfect circle,The Nurse who Loved me (cover)
...,...,...
768,yiruma,River Flows in You
769,yiruma,Sunbeams... They Scatter
770,yo la tengo,My Little Corner of the World
771,beabadoobee,glue song


In [26]:
# truncate 'the '
df['artist'] = df['artist'].apply(lambda s: s[4:] if s[:4]=='the ' else s)
df.loc[df['artist']=='band', 'artist']='the band'
# fix this travesty
df.loc[(df['artist']=='beatles') & (df['track']=='God Only Knows'), 'artist'] ='beach boys'


In [27]:
df.groupby('artist') \
    .count() \
    .reset_index() \
    .sort_values('track', ascending=False) \
    .head(20)


,artist,track
36,beatles,27
301,radiohead,20
191,jim croce,16
286,pink floyd,14
329,simon and garfunkel,14
197,john denver,11
34,beach boys,11
328,simon & garfunkel,10
327,sigur rós,10
83,cure,9


In [28]:
unique_artists = df.groupby('artist') \
    .count() \
    .reset_index() \
    .sort_values('track', ascending=False)

unique_artists.columns=['artist', 'count']
unique_artists

,artist,count
36,beatles,27
301,radiohead,20
191,jim croce,16
286,pink floyd,14
329,simon and garfunkel,14
...,...,...
163,honeydrippers,1
162,hollies,1
161,herb alpert and the tijuana brass,1
160,henry mancini,1


In [29]:
# will need to go through 20 examples or so of true positives and false positives
dedupe_df = pandas_dedupe.dedupe_dataframe(unique_artists, ['artist'])


Importing data ...
Reading from dedupe_dataframe_learned_settings
Clustering...


/var/folders/6d/3xz907yn5ylg43s2vlnnzptr0000gn/T/ipykernel_43853/3429850719.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dedupe_df = pandas_dedupe.dedupe_dataframe(unique_artists, ['artist'])


# duplicate sets 335


In [ ]:
dedupe_df.head(20)

In [30]:
# map to most popular version
unique_artists['cluster id'] = dedupe_df['cluster id']
name2i = {a: i for i, a in zip(unique_artists['cluster id'].tolist(), unique_artists['artist'].tolist())}

df['artist_index'] = df['artist'].apply(lambda s: name2i[s])
df


,artist,track,artist_index
0,hack//sign,Key of the Twilight,157
1,a perfect circle,Gravity,35
2,a perfect circle,Judith,35
3,a perfect circle,The Noose,35
4,a perfect circle,The Nurse who Loved me (cover),35
...,...,...,...
768,yiruma,River Flows in You,333
769,yiruma,Sunbeams... They Scatter,333
770,yo la tengo,My Little Corner of the World,334
771,beabadoobee,glue song,64


In [31]:
# update artist name
tempdf = dedupe_df[['artist', 'cluster id']] \
    .groupby('cluster id') \
    .first() \
    .reset_index()

i2name = {i: a for i, a in zip(tempdf['cluster id'].tolist(), tempdf['artist'].tolist())}
df['artist'] = df.apply(lambda r: i2name[r.artist_index], axis=1)


df

,artist,track,artist_index
0,hack//sign,Key of the Twilight,157
1,a perfect circle,Gravity,35
2,a perfect circle,Judith,35
3,a perfect circle,The Noose,35
4,a perfect circle,The Nurse who Loved me (cover),35
...,...,...,...
768,yiruma,River Flows in You,333
769,yiruma,Sunbeams... They Scatter,333
770,yo la tengo,My Little Corner of the World,334
771,beabadoobee,glue song,64


In [32]:
df.groupby('artist') \
    .count() \
    .reset_index() \
    .sort_values('track', ascending=False) \
    .head(20)


,artist,track,artist_index
33,beatles,27,27
293,simon and garfunkel,24,24
268,radiohead,20,20
292,sigur ros,19,19
169,jim croce,16,16
256,pink floyd,14,14
32,beach boys,14,14
174,john denver,11,11
115,fleetwood mac,10,10
74,cure,9,9


In [37]:
df.groupby('track') \
    .count() \
    .reset_index() \
    .sort_values('artist', ascending=False) \
    .head(20)


,track,artist,artist_index
361,no artist and track mentioned in this post,7,7
366,no track mentioned in this post,7,7
136,dream a little dream of me,5,5
616,what a wonderful world,4,4
368,nothing compares 2 u,4,4
294,landslide,4,4
287,la vie en rose,4,4
211,hallelujah,4,4
523,the book of love,3,3
264,into the mystic,3,3


In [36]:
df['track'] = df['track'].apply(fix_leading_trailing)
df = df.drop(df.loc[df['track']=='no track mentioned in this post.'].index)
df = df.drop(df.loc[df['track']=='no artist and track mentioned in this post.'].index)
df = df.drop(df.loc[df['track']=='unknown'].index)
df = df.drop(df.loc[df['track']=='none'].index)
df = df.drop(df.loc[df['track']=='various tracks'].index)
df = df.drop(df.loc[df['track']=='anything'].index)
df = df.drop(df.loc[df['track']==''].index)
df


,artist,track,artist_index
0,hack//sign,key of the twilight,157
1,a perfect circle,gravity,35
2,a perfect circle,judith,35
3,a perfect circle,the noose,35
4,a perfect circle,the nurse who loved me (cover,35
...,...,...,...
768,yiruma,river flows in you,333
769,yiruma,sunbeams... they scatter,333
770,yo la tengo,my little corner of the world,334
771,beabadoobee,glue song,64


In [38]:
df = df.drop_duplicates() \
    .dropna() \
    .sort_values(["artist", "track"])
df


,artist,track,artist_index
1,a perfect circle,gravity,35
2,a perfect circle,judith,35
3,a perfect circle,the noose,35
4,a perfect circle,the nurse who loved me (cover,35
5,a perfect circle,three libras,35
...,...,...,...
766,yeah yeah yeahs,maps,332
767,yiruma,may be,333
768,yiruma,river flows in you,333
769,yiruma,sunbeams... they scatter,333


In [39]:
df[['artist', 'track']].to_csv('silver.csv', index=False)


## Load into a Spotify playlist


In [40]:
client_credentials_manager = SpotifyClientCredentials(client_id=os.getenv('SPOTIFY_CLIENT_ID'), 
                                                      client_secret=os.getenv('SPOTIFY_CLIENT_SECRET'),
                                                      )

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [41]:
# check artists
df = pd.read_csv("silver.csv")
df = df.drop_duplicates() \
    .dropna() \
    .sort_values(["artist", "track"])

dedupe={}
fail_list = []
c=0

for index, artist, title in df.itertuples():
    c+=1
    if dedupe.get(artist):
        continue
    dedupe[artist]=1
    query_str = 'artist:%s' % (artist)
    artist_results = sp.search(q=query_str, type='artist', limit=3, offset=0, market='US')
    artist_names = [artist['name'] for artist in artist_results['artists']['items']]
    if artist_names:
        if artist.lower() != artist_names[0].lower():
            print(c, artist, artist_names)
    else:
        fail_list.append((artist, title))
        print(c, "not found:", artist, "-", title)

# then clean up manually as appropriate

6 a-ha ['Daryl Hall & John Oates', 'a-ha', 'half•alive']
14 al stewart ['Alexander Stewart', 'Al Stewart', 'Alec Lee-Stewart']
22 not found: allison kraus - when you say nothing at all
28 america ['The All-American Rejects', 'America', 'American Authors']
39 antonio carlos jobim ['Antônio Carlos Jobim', 'JOBIM, ANTONIO CARLOS', 'Antonio Carlos Jobim Miucha']
54 barber ['Sam Barber', 'Jill Barber', 'Samuel Barber']
57 beach boys ['The Beach Boys', 'Kalua Beach Boys', 'Al Jardine of The Beach Boys']
70 beatles ['The Beatles', 'The New Beatles', 'The Beatles Complete On Ukulele']
92 beautiful south ['The Beautiful South', 'Beautiful South']
94 beethoven ['Ludwig van Beethoven', 'Camper Van Beethoven', "Beethoven's Wig"]
98 berlin ['Berliner Philharmoniker', 'Berlin', 'Konzerthaus Kammerorchester Berlin']
100 beyonce ['Beyoncé', 'Beyonce Smith', 'Mc Beyonce']
102 not found: bill withers grover washington jr - just the two of us
108 bjork ['Björk', 'Brant Bjork', 'Beatrice Björkman']
133 ca

In [42]:
artist_map = {
"a-ha": 'a-ha',
"adeem": 'Adeem',
"airborne toxic event": 'The Airborne Toxic Event',
"al stewart": 'Al Stewart',
"alan parsons project": 'The Alan Parsons Project',
"alt j": 'alt-J',
"america": 'America',
"antonio carlos jobim": 'Antônio Carlos Jobim',
"appleseed cast": 'The Appleseed Cast',
"babyface": 'Babyface',
"barber": 'Barber',
"beach boys": 'The Beach Boys',
"beatles": 'The Beatles',
"beautiful south": 'The Beautiful South',
"beethoven": 'Ludwig van Beethoven',
"ben gibbard": 'Benjamin Gibbard',
"berlin": 'Berlin',
"beyonce": 'Beyoncé',
"bjork": 'Björk',
"black crowes": 'The Black Crowes',
"boa": 'bôa',
"cardigans": 'The Cardigans',
"cat stevens": 'Yusuf / Cat Stevens',
"chopin": 'Frédéric Chopin',
"chordettes": 'The Chordettes',
"coctaeu twins": 'Cocteau Twins',
"cranberries": 'The Cranberries',
"crosby, stills, nash young": 'Crosby, Stills, Nash & Young',
"csny": 'Crosby, Stills, Nash',
"cure": 'The Cure',
"dark souls": 'Dark Souls',
"dawid podsiadlo": 'Dawid Podsiadło',
"debussy": 'Claude Debussy',
"declan orourke": "Declan O'Rourke",
"doors": 'The Doors',
"dream academy": 'The Dream Academy',
"dvorak": 'Antonín Dvořák',
"dylan": 'Bob Dylan',
"edith piaf": 'Édith Piaf',
"emerson, lake, and palmer": 'Emerson, Lake & Palmer',
"emiliana torini": 'Emiliana Torrini',
"eric johnson": 'Eric Johnson',
"flamingos": 'The Flamingos',
"florence and the machine": 'Florence + The Machine',
"fray": 'The Fray',
"frederic chopin": 'Frédéric Chopin',
"frente": 'Frente!',
"goo goo dolls": 'The Goo Goo Dolls',
"heart": 'Heart',
"helio sequence": 'The Helio Sequence',
"herb alpert and the tijuana brass": 'Herb Alpert & The Tijuana Brass',
"hollies": 'The Hollies',
"honeydrippers": 'The Honeydrippers',
"iron and wine": 'Iron & Wine',
"iron wine": 'Iron & Wine',
"irresistible force": 'The Irresistible Force',
"israel kamakawiwoole": "Israel Kamakawiwo'ole",
"iz": "Israel Kamakawiwo'ole",
"iz kamakawiwo`ole": "Israel Kamakawiwo'ole",
"jason isbell": 'Jason Isbell',
"jewel": 'Jewel',
"joaquin rodrigo": 'Joaquín Rodrigo',
"jose gonzalez": 'José González',
"kings singers": "The King's Singers",
"laura anton": 'Laura & Anton',
"lauryn hill": 'Ms. Lauryn Hill',
"leo delibes": 'Léo Delibes',
"les miserables": 'Les Misérables Cast',
"loggins messina": 'Loggins & Messina',
"magnetic fields": 'The Magnetic Fields',
"mamas the papas": 'The Mamas & The Papas',
"maurice durufle": 'Maurice Duruflé',
"mayer haw": 'Mayer Hawthorne',
"meatloaf": 'Meat Loaf',
"mick hucknall simply red": 'Simply Red',
"monkees": 'The Monkees',
"moody blues": 'The Moody Blues',
"mozart": 'Wolfgang Amadeus Mozart',
"mumford and sons": 'Mumford & Sons',
"nick cave and the bad seeds": 'Nick Cave & The Bad Seeds',
"pachelbel": 'Johann Pachelbel',
"paper kites": 'The Paper Kites',
"postal service": 'The Postal Service',
"righteous brothers": 'The Righteous Brothers',
"rolling stones": 'The Rolling Stones',
"ronettes": 'The Ronettes',
"saint-saens": 'Camille Saint-Saëns',
"sammy davis jr": 'Sammy Davis Jr.',
"seal": 'Seal',
"secret sisters": 'The Secret Sisters',
"selena": 'Selena Gomez',
"sigur ros": 'Sigur Rós',
"simon garfunkel": 'Simon & Garfunkel',
"sinead oconnor": "Sinéad O'Connor",
"smashing pumpkins": 'The Smashing Pumpkins',
"smiths": 'The Smiths',
"sundays": 'The Sundays',
"tchaikovsky": 'Pyotr Ilyich Tchaikovsky',
"the band": 'The Band',
"velvet underground": 'The Velvet Underground',
"verve": 'The Verve',
"yaz": 'Yazoo',
}

df['artist'] = df['artist'].apply(lambda s: artist_map[s] if s in artist_map else s)
df[['artist', 'track']].to_csv('silver.csv', index=False)


In [43]:
# check tracks

df = pd.read_csv("silver.csv")
df.drop_duplicates() \
    .dropna() \
    .sort_values(["artist", "track"])

dedupe = {}
mylist = []
fail_list = []
artist_list, track_list, uri_list, album_list = [], [], [], []
orig_artist, orig_track = [], []

for index, artist, title in df.itertuples():
    query_str = 'artist:%s track:%s' % (artist, title)
    track_results = sp.search(q=query_str, type='track', limit=1, offset=0, market='US')
    results = track_results['tracks']['items']
    
    if results:
        r = results[0]
        # failsafe to never put same track twice
        if dedupe.get(r['id']):
            continue
        dedupe[r['id']]=True
        if title.lower() != r['name'].lower():
            print ("%s|%s : %s|%s" % (artist, title, r['artists'][0]['name'], r['name']))
        uri_list.append(r['uri'])
        artist_list.append(r['artists'][0]['name'])
        track_list.append(r['name'])
        album_list.append(r['album']['name'])
        orig_artist.append(artist)
        orig_track.append(title)
#         print('  ',
#               r['artists'][0]['name'],'|',
#               r['name'], '|',
#               r['album']['name'],'|',
#               r['album']['release_date'],'|',
#               r['popularity'])
    else:
        fail_list.append((artist, title))
        print("not found:", artist, "-", title)

not found: a perfect circle - the nurse who loved me (cover
a perfect circle|three libras : A Perfect Circle|3 Libras
not found: adeem the artist - white trash revelry
al green|still in love with you : Al Green|I'm Still in Love with You
alice in chains|down in a hole : Alice In Chains|Down In A Hole (2022 Remaster)
alice in chains|rain when i die : Alice In Chains|Rain When I Die (2022 Remaster)
not found: allison kraus - when you say nothing at all
not found: alter bridge - no artist and track mentioned in this post
andrea bocelli|con te partiro : Andrea Bocelli|Con te partirò
not found: andrea bocelli - pie jesu
not found: aphex twin - avrl 14
aphex twin|rhubarb : Aphex Twin|Donkey Rhubarb
not found: apocalyptica - very cool blend of classical and heavy metal
art garfunkel|bridge over troubled water : Art Garfunkel jr.|Geh mit mir durch den Regenbogen (Bridge Over Troubled Water)
not found: aurora - any song
not found: aurora - teardrop
Barber|adagio for strings : Samuel Barber|Barb

not found: etta james - i'd rather go blind
not found: etta james - i’d rather go blind
etta james|sunday kind of love : Etta James|A Sunday Kind Of Love
not found: eva cassidy - no track mentioned in this post
not found: eva cassidy - open arms
evanescence|even in death : Evanescence|Even In Death - 2016 Version
ewan mcgregor|your song : Ewan McGregor|Your Song - From "Moulin Rouge" Soundtrack
not found: final fantasy x - road to zanarkand
flaming lips|do you realize : The Flaming Lips|Do You Realize??
not found: flaming lips - only have eyes for you
fleetwood mac|albatross : Fleetwood Mac|Albatross - 2018 Master
not found: fleetwood mac - duvet
fleetwood mac|everywhere : Fleetwood Mac|Everywhere - 2017 Remaster
not found: fleetwood mac - helplessness blues
fleetwood mac|silver springs : Fleetwood Mac|Silver Springs - 2004 Remaster
fleetwood mac|songbird : Fleetwood Mac|Songbird - 2004 Remaster
not found: fleetwood mac - the shrine / an argument
not found: frames - eyes open
not found

patsy cline|i fall to pieces : Patsy Cline|I Fall To Pieces - Single Version
paul mccartney|here today : Paul McCartney|Here Today - Remixed 2015
peter gabriel|in your eyes : Peter Gabriel|In Your Eyes - 2012 Remaster
peter gabriel|mercy street : Peter Gabriel|Mercy Street - 2012 Remaster
not found: peter gabriel - salisbury hill
not found: pharaoh sanders - love is everywhere
pink floyd|breathe : Pink Floyd|Breathe (In the Air)
pink floyd|great gig in the sky : Pink Floyd|The Great Gig in the Sky
The Postal Service|such great heights : The Postal Service|Such Great Heights - Remastered
not found: promise - when in rome
puccini|un bel di and vissi d’arte : Giacomo Puccini|Tosca, S.69, IGP 17, Act II: "Vissi d'arte, vissi d'amore" (Tosca)
queen|good old fashioned lover boy : Queen|Good Old-Fashioned Lover Boy - Remastered 2011
queen|love of my life : Queen|Love Of My Life - Remastered 2011
not found: queen - no track mentioned in this post
queen|scarborough fair : Queensrÿche|Scarboroug

In [44]:
gold_df = pd.DataFrame({'input_artist': orig_artist,
                        'artist': artist_list,
                        'input_track': orig_track,
                        'track': track_list,
                        'album': album_list,
                        'uri': uri_list})
gold_df

,input_artist,artist,input_track,track,album,uri
0,a perfect circle,A Perfect Circle,gravity,Gravity,Thirteenth Step,spotify:track:1CO4BB8CaiQggtJ0R6GwGt
1,a perfect circle,A Perfect Circle,judith,Judith,Mer De Noms,spotify:track:5KDNFlHAdDJ84fhK27c35X
2,a perfect circle,A Perfect Circle,the noose,The Noose,Thirteenth Step,spotify:track:6lvNLD1XRU5paMwWH0RGRI
3,a perfect circle,A Perfect Circle,three libras,3 Libras,Mer De Noms,spotify:track:5kHkaBN8OEQlmXfQkACxSt
4,a-ha,a-ha,take on me,Take on Me,Hunting High and Low,spotify:track:2WfaOiMkCvy7F5fcp2zZ8L
...,...,...,...,...,...,...
564,yeah yeah yeahs,Yeah Yeah Yeahs,maps,Maps,Fever To Tell (Deluxe Remastered),spotify:track:0hDQV9X1Da5JrwhK8gu86p
565,yiruma,Yiruma,may be,May Be,First Love (The Original & the Very First Reco...,spotify:track:42WpwYm7DYxbbVbUO35LUX
566,yiruma,Yiruma,river flows in you,River Flows In You,First Love (The Original & the Very First Reco...,spotify:track:2agBDIr9MYDUducQPC1sFU
567,yiruma,Yiruma,sunbeams... they scatter,"The Sunbeams,They Scatter",SOLO,spotify:track:0pjMeOto2TvTDJheqQxpaH


In [45]:
# non matching
with pd.option_context("display.max_rows", 999):
    display(gold_df.loc[gold_df['input_artist'].str.lower() != gold_df['artist'].str.lower()])

,input_artist,artist,input_track,track,album,uri
18,alison krauss,Alison Krauss & Union Station,the lucky one,The Lucky One,New Favorite,spotify:track:6xcSL2he3cuh2OSkAlbubf
40,art garfunkel,Art Garfunkel jr.,bridge over troubled water,Geh mit mir durch den Regenbogen (Bridge Over ...,Wie Du: Hommage an meinen Vater (Zweite Edition),spotify:track:0WxC7uOKJh0Dcn2by2onot
44,Barber,Samuel Barber,adagio for strings,Barber: Adagio for Strings,Samuel Barber - Adagio,spotify:track:1CSaCKPIp2yCIDL3t7Fyau
73,The Beautiful South,The Karaoke Channel,dream a little dream of me,Dream A Little Dream Of Me [In the Style of Be...,The Karaoke Channel - The Best Of Pop Vol. - 31,spotify:track:3kx08p814N09kHRfA728dZ
98,brand new,Brand New Key,the story,End of the Story,End of the Story,spotify:track:39yaRdiL4NG3qk3iAh0c8b
111,cat stevens (yusuf islam,Yusuf / Cat Stevens,the wind,The Wind - Remastered 2021,Teaser And The Firecat (Remastered 2021),spotify:track:0wEo4mwfs6tXYgCem9m9wP
157,dixie chicks,Karaoke - Dixie Chicks,landslide,Karaoke - Landslide,Karaoke - Contemporary Female Country - Vol.4,spotify:track:38wTBBl2xMxaSP67S5cin3
167,ella fitzgerald and louis armstrong,Ella Fitzgerald,summertime,Summertime,Porgy And Bess,spotify:track:2gNjmvuQiEd2z9SqyYi8HH
173,elvis,Elvis Presley,can’t help falling in love,Can’t Help Falling in Love - Acoustic Cover,Music for the Moment: Evening Bath with Classi...,spotify:track:0ghQkNDYLSl4GsqfkjTjWx
174,elvis,Elvis Presley,unchained melody,"Unchained Melody - Live at Ann Arbor, MI",Moody Blue,spotify:track:0OavtQSojULqejmC4Qbstr


In [46]:
# clean up gold3 and add the ones which are not already in the playlist


In [ ]:
gold_df2=gold_df.copy().reset_index(drop=True)
gold_df2['input_track']=gold_df2['input_track'].str.lower()
gold_df2['input_track']=gold_df2['input_track'].apply(lambda s: s.strip()[:10])

gold_df2['track']=gold_df2['track'].str.lower()
gold_df2['track']=gold_df2['track'].apply(lambda s: s.strip()[:10])


with pd.option_context("display.max_rows", 999):
    display(gold_df2.loc[gold_df2['input_track'] != gold_df2['track']])

In [ ]:
# these are songs that look like covers or otherwise not the expected response from spotify search 
# (which is a bit wonky, doesn't like quotes and such)

bad_lookups = [
#    25,134,155,160,200,209,422,445,446,557,737,744,755,759,760,761,762,781,785,790,814,815,842
    21,51,61,63,83,145,212,317,322,439,449,575,759,784,
]

for i in bad_lookups:
    print(gold_df.iloc[i])
    
# add manually, plus 'not found'


In [ ]:
gold_df = gold_df.drop(
    axis='index',
    labels=bad_lookups)

gold_df[['artist', 'track']].to_csv('gold.csv', index=False)

with pd.option_context("display.max_rows", 999):
    display(gold_df)

In [ ]:
# get playlist id
# first create a playlist in UI to load songs
playlists = sp.user_playlists(os.getenv('SPOTIFY_USERNAME'))
while playlists:
    for i, playlist in enumerate(playlists['items']):
        if playlist['name'] != 'Reddit Prettiest Songs':
            continue
        print(playlist['id'])
        playlist_id = playlist['id']
        print("%4d %s %s" % (i + 1 + playlists['offset'], playlist['uri'],  playlist['name']))
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

In [ ]:
# must follow an oauth workflow to write a playlist in Spotify
# running this cell should request a spotify login and then redirect to an url
# paste whole url with id into form to authenticate

scope = "playlist-modify-public"

sp = spotipy.Spotify(auth_manager=spotipy.SpotifyOAuth(scope=scope,
                                                       client_id=os.getenv('SPOTIFY_CLIENT_ID'),
                                                       client_secret=os.getenv('SPOTIFY_CLIENT_SECRET'),
                                                       redirect_uri="https://druce.ai"
                                                      ))

In [ ]:
# addlist = gold_df['uri'].to_list()
# print (len(addlist))

# while(addlist):
#     sp.user_playlist_add_tracks(os.getenv('SPOTIFY_USERNAME'), 
#                                 playlist_id=playlist_id, 
#                                 tracks=addlist[-100:])
#     addlist = addlist[:-100]
#     print("added items, remaining ", len(addlist))


In [ ]:
# manually add the ones that weren't found for some reason


In [ ]:
# can run again and add any new tracks, either because OpenAI is a bit random, or new replies in thread
results = sp.user_playlist(os.getenv('SPOTIFY_USERNAME'), playlist_id,
                                fields='tracks,next,name')
tracks = results['tracks']

playlist_dict_by_uri = {}
playlist_dict_by_str = {}

artist_list = []
track_list = []
uri_list = []
popularity_list = []
album_list=[]

while True:
    for track_item in tracks['items']:
        track_dict = track_item['track']
        track_str = track_dict['artists'][0]['name']  + ' | ' + track_dict['name'][:15]
        uri = track_dict['uri']
        if track_str in playlist_dict_by_str:
            print(track_str)
        playlist_dict_by_str[track_str] = uri
        playlist_dict_by_uri[uri] = track_str
        
        uri_list.append(uri)
        artist_list.append(track_dict['artists'][0]['name'])
        track_list.append(track_dict['name'])
        album_list.append(track_dict['album']['name'])
        popularity_list.append(track_dict['popularity'])
        
    # check if there are more pages
    if tracks['next']:
        tracks = sp.next(tracks)
    else:
        break

print (len(list(playlist_dict_by_str.keys())))
print (len(list(playlist_dict_by_uri.keys())))


In [ ]:
playlist_df = pd.DataFrame({'artist': artist_list,
                           'track': track_list,
                           'album': album_list,
                           'popularity': popularity_list,
                           })



In [ ]:
with pd.option_context("display.max_rows", 9999):
    display(playlist_df.sort_values('popularity'))
    

In [ ]:
gold_dict_by_uri = {}
gold_dict_by_str = {}
addlist = []
c = 0
for i, artist, track, uri in gold_df[['artist', 'track', 'uri']].itertuples():
    # print(artist, track, uri)
    track_str = artist + ' | ' + track[:15]
    if track_str not in playlist_dict_by_str:
        addlist.append([artist, track, uri])
        print(artist, track, uri)
    gold_dict_by_uri[uri]=track_str
    gold_dict_by_str['track_str']= uri
#     if track_str not in playlist_dict_by_str:
#         c += 1
#         print (c, track_str)
        
print(len(gold_dict_by_str.items()))
print(len(gold_dict_by_uri.items()))

In [ ]:
addlist

In [ ]:
addlist = [['ABBA', 'One Of Us', 'spotify:track:6zgtBUEkAfilJ2YEOvNexR'],
 ['Gregorio Allegri',
  'Miserere mei, Deus',
  'spotify:track:6es7DmrhnDoKj5rsFvh3XU'],
 ['Amy Winehouse',
  'Love Is A Losing Game',
  'spotify:track:3uliGwmB52ZA7brgpZMzyH'],
 ['Barbara',
  "Ma plus belle histoire d'amour",
  'spotify:track:0qBVET4VkHsQAoboWlQ2pJ'],
 ['Ludwig van Beethoven',
  'Symphony No. 5 in C Minor, Op. 67: I. Allegro con brio',
  'spotify:track:2ygeBLTP9uu3OW3VTulD8N'],
 ['Benny Goodman', 'Sing, Sing, Sing', 'spotify:track:5L8ta4ECl5zeA6bGqY7G38'],
 ['Bill Withers', 'Lean on Me', 'spotify:track:3M8FzayQWtkvOhqMn2V4T2'],
 ['Billy Joel', 'Piano Man', 'spotify:track:70C4NyhjD5OZUMzvWZ3njJ'],
 ['Bob Dylan', 'Ballad of a Thin Man', 'spotify:track:0f5N14nB8xi0p3o4BlVvbx'],
 ['Bob Dylan', "Blowin' in the Wind", 'spotify:track:18GiV1BaXzPVYpp9rmOg0E'],
 ['Bob Dylan', 'Desolation Row', 'spotify:track:4n1ZGm3TxYmoYe1YR8cMus'],
 ['Bob Dylan', 'Duquesne Whistle', 'spotify:track:5kKW4bszhKSCYVPDO0sMbX'],
 ['Bob Dylan',
  'Forever Young - Slow Version',
  'spotify:track:4yWl0tnEanf3zmZzl9kbQn'],
 ['Bob Dylan', 'Gotta Serve Somebody', 'spotify:track:760420tYNmNjFgi8bWvbop'],
 ['Bob Dylan', 'Highway 61 Revisited', 'spotify:track:6os5B6xjuke9YfBKH3tu1e'],
 ['Bob Dylan',
  'I Shall Be Released - Studio Outtake - 1971',
  'spotify:track:5vyw005QQ42hrzrLxb3xEX'],
 ['Bob Dylan', 'I Want You', 'spotify:track:7tJQ4Ekp2vN3NlI3vJJW3v'],
 ['Bob Dylan', "It Ain't Me Babe", 'spotify:track:5nbNWAfT1S6V1vqj3snHxS'],
 ['Bob Dylan', 'Jokerman', 'spotify:track:6cuHkcRUqtQhtJ4sWCkd1q'],
 ['Bob Dylan',
  "Knockin' On Heaven's Door",
  'spotify:track:6HSXNV0b4M4cLJ7ljgVVeh'],
 ['Bob Dylan', 'Lay, Lady, Lay', 'spotify:track:4uYwlMp841PLJmj1gJJwIq'],
 ['Bob Dylan', 'Like a Rolling Stone', 'spotify:track:3AhXZa8sUQht0UEdBJgpGc'],
 ['Bob Dylan', 'Love Sick', 'spotify:track:3O1hpSOaJDW4SelgUG2XT3'],
 ['Bob Dylan', "Maggie's Farm", 'spotify:track:5rGD8FFgHw74cp3RPhucyg'],
 ['Bob Dylan',
  'Make You Feel My Love',
  'spotify:track:6rfGPGghQL7SJmZPXprXIc'],
 ['Bob Dylan',
  'Mississippi - Version 2',
  'spotify:track:6JWHNd8QMxTvojYkmZtKGI'],
 ['Bob Dylan', 'Mr. Tambourine Man', 'spotify:track:3RkQ3UwOyPqpIiIvGVewuU'],
 ['Bob Dylan', 'Murder Most Foul', 'spotify:track:1LfTvT9JPYuuZanwxLtZCr'],
 ['Bob Dylan', 'Not Dark Yet', 'spotify:track:1qbn6QrHG8XfnqVFKgNzKP'],
 ['Bob Dylan',
  'Rainy Day Women #12 & 35',
  'spotify:track:7BkAlVpGwXXl3sYNn5OoJ7'],
 ['Bob Dylan',
  'Sad-Eyed Lady of the Lowlands',
  'spotify:track:4jdtLLyEL7wY0TlCdMKhxq'],
 ['Bob Dylan', 'She Belongs to Me', 'spotify:track:2itBkHBUxGl4VfDj4HNyoD'],
 ['Bob Dylan',
  'Stuck Inside of Mobile with the Memphis Blues Again',
  'spotify:track:1NYTj6JEw3IOh4ggiBh82h'],
 ['Bob Dylan',
  'Subterranean Homesick Blues',
  'spotify:track:6k9DUKMJpWvu6eFG3O64Lg'],
 ['Bob Dylan', 'Tangled up in Blue', 'spotify:track:6Vcwr9tb3ZLO63F8DL8cqu'],
 ['Bob Dylan', 'Tempest', 'spotify:track:19scNzd4ogVsHrNWsms8Rg'],
 ['Bob Dylan',
  "The Times They Are A-Changin'",
  'spotify:track:52vA3CYKZqZVdQnzRrdZt6'],
 ['Bob Dylan',
  'Things Have Changed - Single Version',
  'spotify:track:5KOi77ameCimkAdw0DMNoy'],
 ['Bob Dylan',
  'Thunder on the Mountain',
  'spotify:track:4wo2eRp6aHcAlmhmfwiTAH'],
 ['Bob Dylan', 'Visions of Johanna', 'spotify:track:2rslQV48gNv3r9pPrQFPW1'],
 ['Brian Wilson', 'God Only Knows', 'spotify:track:2SznAUigFh6rMdGpcS5d7e'],
 ['Bright Eyes',
  'First Day of My Life',
  'spotify:track:0eBryM7ePQH3Klt3jz8xZd'],
 ['Crowded House',
  'Don’t Dream It’s Over - Home Demo',
  'spotify:track:0fiSpF9mvRFQWy0ca64d1g'],
 ['Léo Delibes', 'Flower Duet', 'spotify:track:5K8jqeLAxZIqHR6e5w5so1'],
 ['Dire Straits', 'Brothers In Arms', 'spotify:track:6XYBbVpu455ZdGWZNRLGbG'],
 ['Don McLean',
  'Vincent (Starry, Starry Night)',
  'spotify:track:2YDyH60Vro33KkDtNZCXIk'],
 ['Ed Sheeran', 'Photograph', 'spotify:track:41xNsY82OWtWbIfnRMK2ky'],
 ['Elvis Presley',
  'Can’t Help Falling in Love - Acoustic Cover',
  'spotify:track:0ghQkNDYLSl4GsqfkjTjWx'],
 ['Enya', 'Amarantine', 'spotify:track:0VmzazQQ0Mo1vJldr5NxTW'],
 ['Evan Rachel Wood', 'If I Fell', 'spotify:track:0gd3hRBQAEAw096YOcUrmR'],
 ['Fleetwood Mac', 'Rhiannon', 'spotify:track:05oETzWbd4SI33qK2gbJfR'],
 ['George Harrison',
  'All Things Must Pass - 2014 Remaster',
  'spotify:track:16OwZQuzMqnwn3FZsCBZly'],
 ['George Harrison',
  'Apple Scruffs - 2014 Remaster',
  'spotify:track:2K7WhpfZX3TCCMiwebp0W7'],
 ['George Harrison',
  'Art of Dying - 2014 Remaster',
  'spotify:track:6Jod7qrtYBhU3HcUmKk4hX'],
 ['George Harrison',
  'Awaiting on You All - 2014 Remaster',
  'spotify:track:0b65WkrBrg2qOkzQeDtQ9d'],
 ['George Harrison',
  'Ballad of Sir Frankie Crisp (Let It Roll) - 2014 Remaster',
  'spotify:track:0FWeRrB8T5R6maHbWQw4Kk'],
 ['George Harrison',
  'Behind That Locked Door',
  'spotify:track:2VVbLn8nMcWJzjcL1tZsUr'],
 ['George Harrison',
  'Beware of Darkness - 2014 Remaster',
  'spotify:track:606MCyZFMBlc52Ojnn1nvU'],
 ['George Harrison',
  'Give Me Love (Give Me Peace on Earth)',
  'spotify:track:71fXxvXqo1zxWDtBmjoEVk'],
 ['George Harrison',
  'Hear Me Lord - 2014 Remaster',
  'spotify:track:3kopbNyRj10XO1actGZexP'],
 ['George Harrison',
  'I Dig Love - 2014 Remaster',
  'spotify:track:42yK1Wy62c7malKSRwy0Qk'],
 ['George Harrison',
  'I Remember Jeep - 2014 Remaster',
  'spotify:track:058AE5M3ifbCh8VWOV7903'],
 ['George Harrison',
  "It's Johnny's Birthday - 2014 Remaster",
  'spotify:track:6Cv05rcW8HWwCC6wyEp1fC'],
 ['George Harrison',
  'Let It Down - 2014 Remaster',
  'spotify:track:5FFruMKbVg8AhwHnX4xBov'],
 ['George Harrison',
  'My Sweet Lord - 2014 Remaster',
  'spotify:track:6vE90mi4yKsQGY3YD2OOv1'],
 ['George Harrison',
  'Out of the Blue - 2014 Remaster',
  'spotify:track:1KHMyFaGvwVQ7ax4yjq4BZ'],
 ['George Harrison',
  'Plug Me In - 2014 Remaster',
  'spotify:track:0tyk2xHVjBd3nk16cGktTG'],
 ['George Harrison',
  'Run of the Mill - 2014 Remaster',
  'spotify:track:4uSlUBg3NVOA77E7wwKFTO'],
 ['George Harrison',
  'Thanks for the Pepperoni - 2014 Remaster',
  'spotify:track:3smkwfPqFsTmwfnBztMXaM'],
 ['George Harrison',
  'The Inner Light (Alternative Take) - Instrumental',
  'spotify:track:7gWPnvhaBFMlQsTBWEGcSC'],
 ['George Harrison',
  'Wah-Wah - 2014 Remaster',
  'spotify:track:5j3aqkMO2fl0s5eaSuVnQ8'],
 ['George Harrison',
  'What Is Life - 2014 Remaster',
  'spotify:track:44fw7RulJyj7dGIi9qR86N'],
 ['George Harrison',
  'While My Guitar Gently Weeps - Live At Madison Square Garden; 2009 Remaster',
  'spotify:track:4Egi6XuC0rbLlXfqmQeuFa'],
 ['Glenn Miller', 'In the Mood', 'spotify:track:1xsY8IFXUrxeet1Fcmk4oC'],
 ['Hans Zimmer', 'Cornfield Chase', 'spotify:track:6pWgRkpqVfxnj3WuIcJ7WP'],
 ['Hans Zimmer',
  'Day One (Interstellar Theme)',
  'spotify:track:4WmB04GBqS4xPMYN9dHgBw'],
 ["Israel Kamakawiwo'ole",
  'Maui Medley',
  'spotify:track:6TSJ3L9pBQsYIlCD5pk7ju'],
 ['James Taylor',
  'You’ve Got a Friend',
  'spotify:track:3nK4hWsTEr7fVXziI5bTmh'],
 ['Jay Ungar', 'Ashoken Farewell', 'spotify:track:2s6pqLeVialgt5l5TTSeas'],
 ['Jeff Buckley',
  'If You Knew - Live at Sin-é, New York, NY - July/August 1993',
  'spotify:track:1nd2JEHXbUuQFDiQzCBpsv'],
 ['Jimi Hendrix', 'One Rainy Wish', 'spotify:track:5Zyv0v4rPcrXjkaeImuodv'],
 ['Jimi Hendrix',
  'Spanish Castle Magic',
  'spotify:track:2KFE98Iw0X23sf4vJYcbLH'],
 ['Jimi Hendrix',
  'Wait Until Tomorrow',
  'spotify:track:2YtVzmZzew1ILUdNueyWd7'],
 ['John Lennon',
  'Imagine - Remastered 2010',
  'spotify:track:7pKfPomDEeI4TPT6EOYjn9'],
 ['John Mayer', 'Queen of California', 'spotify:track:0CETmgFGt8Ne8vLnaLcduU'],
 ['Johnny Cash',
  'I Walk The Line - Single Version',
  'spotify:track:1TKPfF2fvn6gVLVfp3iG4j'],
 ['Joni Mitchell',
  'Mitchell: Urge for Going (Instrumental Arrangement of the B-Side Track of the Joni Mitchell Single "You Turn Me on I\'m a Radio")',
  'spotify:track:1I1u9aTdxxQ7SDLgBB3V7b'],
 ['Kanye West', 'Come to Life', 'spotify:track:5xvXeuxISyXJDRbZZf4uzd'],
 ['Leonard Cohen', 'Chelsea Hotel #2', 'spotify:track:4krhCfJg0znykZoyjeMXRe'],
 ['Leonard Cohen', 'Dear Heather', 'spotify:track:3MTKMphPprAcBFG1uIhzPZ'],
 ['Leonard Cohen',
  "Death of a Ladies' Man",
  'spotify:track:5wrylUGwZugelovhryPYg2'],
 ['Leonard Cohen', 'The Future', 'spotify:track:5l8lYrnPEM1ln3J4XaTcy5'],
 ['Leonard Cohen',
  'You Want It Darker',
  'spotify:track:5zb7npjQqoJ7Kcpq4yD9qn'],
 ['Lingers.On', 'In Lingerie', 'spotify:track:6FH3kGlJbFVJDCG9RcERf7'],
 ['Louis Armstrong',
  'La vie en rose - Single Version',
  'spotify:track:3yYfoYGVpriV4fG9L1ogsD'],
 ['The Lovecats', 'The Lovecats', 'spotify:track:7iJUiiTfnuY5cTIeEBnqHr'],
 ['Ludovico Einaudi', 'Primavera', 'spotify:track:4BMHp3DkI8VLsuB9Kr0pzu'],
 ['Mazzy Star', 'Flowers In December', 'spotify:track:0G6Ws8Gbdt0S7pZeuYmkmm'],
 ['Metallica',
  'Fade To Black (Remastered)',
  'spotify:track:0dqGfCMAGyDgpUAgLNOjWd'],
 ['Wolfgang Amadeus Mozart',
  'Requiem in D Minor, K. 626: III. Sequenz No. 6, Lacrimosa dies illa',
  'spotify:track:4bvzJZXpkI3bkjxMCWOSu1'],
 ['My Chemical Romance',
  'The Light Behind Your Eyes',
  'spotify:track:3HyDpKAuR3e4l6QB7hSB2l'],
 ['Paul McCartney',
  'Here Today - Remixed 2015',
  'spotify:track:0QtnwXDziZN1K55fXuLN6q'],
 ['Paul McCartney',
  'I’ll Follow The Sun - Live At Amoeba 2007',
  'spotify:track:3xT59EeQdq0TPGtOlXXI8t'],
 ['Puscifer', 'The Humbling River', 'spotify:track:69GE6yPZZldvqtgBHrKXxg'],
 ['Ray LaMontagne',
  'Such A Simple Thing',
  'spotify:track:4PuUa8e5s7P3Zv1IdCGIsa'],
 ['Ray Manzarek',
  'Riders on the Storm',
  'spotify:track:3FvYcTXO2QtDY7kZQHku2d'],
 ['Red Hot Chili Peppers', 'Dosed', 'spotify:track:1iFIZUVDBCCkWe705FLXto'],
 ['Sky Cries Mary',
  "Don't Forget The Sky",
  'spotify:track:4sVpjCJRClVetRrdxVBolP'],
 ['Stevie Nicks', 'Landslide', 'spotify:track:5fprEY6WEN1wvFXkgfb22C'],
 ['Stevie Wonder', 'Isn’t She Lovely', 'spotify:track:6wGlAaMfyhKdEPr2zycAnN'],
 ['Taylor Swift',
  'Fearless (Taylor’s Version)',
  'spotify:track:77sMIMlNaSURUAXq5coCxE'],
 ['Taylor Swift',
  'the lakes - bonus track',
  'spotify:track:0eFQWVz0qIxDOvhLpZ40P7'],
 ['The Band',
  'When I Paint My Masterpiece - Remastered',
  'spotify:track:76WChUuOPeIK027IeUgr0l'],
 ['The Beach Boys',
  "I Just Wasn't Made For These Times - Mono",
  'spotify:track:4CuO8TINNqM3D7aUdNQ3zG'],
 ['The Beach Boys',
  "Let's Go Away For A While - Mono",
  'spotify:track:3GsgJI1aBrvUtqX8f3MhKT'],
 ['The Beatles',
  "Don't Let Me Down - Naked Version / Remastered 2013",
  'spotify:track:5BhMoGrz5KzG2fA5uzHjZ1'],
 ['The Beatles',
  'Love Me Do - Remastered 2009',
  'spotify:track:3VbGCXWRiouAq8VyMYN2MI'],
 ['The Chemical Brothers',
  'The Boxer',
  'spotify:track:1EUeDFq2zNP784GPaRs9aH'],
 ['The Cure',
  'A Night like This - 2006 Remaster',
  'spotify:track:7cKCz7gG84i1XLvDeM3ByT'],
 ['The Cure',
  'Disintegration - 2010 Remaster',
  'spotify:track:0zY8t5dC1KQXcPUKByWMJM'],
 ['The Cure',
  'From the Edge of the Deep Green Sea',
  'spotify:track:2vwBL9RVyr0vA4Og5VH0i3'],
 ['The Cure',
  'In Between Days - 2006 Remaster',
  'spotify:track:07CyrZF9eVd02zzIse7tZA'],
 ['The Cure', 'A Letter to Elise', 'spotify:track:4DdXOLc1VMAY34ourCn1Xa'],
 ['The Cure',
  'Lullaby - 2010 Remaster',
  'spotify:track:4d4oXk7O2lEhZ83ivV93li'],
 ['The Cure', 'Underneath The Stars', 'spotify:track:0PKVjYlKw7z3IvKAoxrYTR'],
 ['The Eagles', 'The Desperadoes', 'spotify:track:10ppF835WJMYI5v65gFLZ3'],
 ['The Helio Sequence',
  'Keep Your Eyes Ahead',
  'spotify:track:3yatRBsGMJ7wMoUIgDBzzo'],
 ['The Moldy Peaches',
  'Anyone Else But You',
  'spotify:track:2pKi1lRvXNASy7ybeQIDTy'],
 ['The Strokes', 'Someday', 'spotify:track:7hm4HTk9encxT0LYC0J6oI'],
 ['Traditional',
  'Scarborough Fair (Arr. Parkin)',
  'spotify:track:4wlNPczIullwvmwb4x0ltz'],
 ['Van Morrison',
  'Madame George - 1999 Remaster',
  'spotify:track:1N4MKISvC1ddfRCRQDXDd2'],
 ['Various Artists',
  'The Girl From Ipanema',
  'spotify:track:0JgH7g0kwsIs1THEVqhlUS'],
 ['Víg Mihály',
  'Öreg - From "Werckmeister Harmóniák"',
  'spotify:track:63wMgkXQuomlkW4an4O9b4'],
 ['Willie Nelson', 'Crazy', 'spotify:track:0xqtcLB45iKNfHroi5y1em']]


In [ ]:
len(addlist)

In [ ]:
addlist2 = [a[2] for a in addlist]

print (len(addlist2), 'items')

while(addlist2):
    sp.user_playlist_add_tracks(os.getenv('SPOTIFY_USERNAME'), 
                                playlist_id=playlist_id, 
                                tracks=addlist2[-100:])
    addlist2 = addlist2[:-100]
    print("added items, remaining ", len(addlist2))
